In [1]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import astropy
import scipy
from filterpy.kalman import KalmanFilter 
from filterpy.common import Q_discrete_white_noise
from scipy.linalg import block_diag
from filterpy.common import Saver
from astropy import units as u
import glob
# Own Libraries
from utility.utils import *
from KalmanFilter.kf import *
from Match.pair import *
from Match.icp import *

%matplotlib tk
style.use('seaborn-paper')

In [2]:
############################################
dt = 10
############################################
df = pd.read_csv(f'DATA/ephemeris sat/inclination zero/{dt} step size.csv', header=3, sep=';') 
real_Latitudes, real_Longitudes, real_Altitudes = df['Lat (deg)'], df['Lon (deg)'], df['Alt (km)']
real_Vxs,real_Vys,real_Vzs = df['x (km/sec)'], df['y (km/sec)'],df['z (km/sec)']

real_X, real_Y, real_Z = [], [], []
for i in range(len(df)):
    altitude = real_Altitudes[i]
    latitude = real_Latitudes[i]
    longitude = real_Longitudes[i]
    x, y, z = spherical2cartesian(altitude, latitude, longitude)
    real_X.append(x)
    real_Y.append(y)
    real_Z.append(z)
real_X, real_Y, real_Z = np.array(real_X),np.array(real_Y),np.array(real_Z)

In [3]:
# plt.figure(dpi=150, tight_layout=True)

# sub_x = plt.subplot(3,1,1)
# plt.title('Satellite True Position')
# plt.plot(real_X, 'k', linewidth=0.8)
# plt.ylabel('X')

# sub_y = plt.subplot(3,1,2, sharex = sub_x)
# plt.plot(real_Y,'k',linewidth=0.8)
# plt.ylabel('Y')

# sub_z = plt.subplot(3,1,3, sharex = sub_x)
# plt.plot(real_Z,'k',linewidth=0.8)
# plt.xlabel('Time step (0.1 sec)')
# plt.ylabel('Z')
# plt.ylim([-1,1])

# plt.show()

In [4]:
# Initial State Vector:
x, y, z = real_X[0], real_Y[0], real_Z[0]
vx, vy, vz = real_Vxs[0], real_Vys[0], real_Vzs[0]

In [18]:
kf = KalmanFilter(dim_x = 6, dim_z = 3)
# Offset State Vector:
iX, iY, iZ, iVx, iVy, iVz  = 0,1,2,3,4,5
kf.F[iX, iVx] = dt
kf.F[iY, iVy] = dt
kf.F[iZ, iVz] = dt
q = Q_discrete_white_noise(dim=3, dt=dt, var=0.1)
kf.Q = block_diag(q, q)
kf.Q*=1e10
kf.H[iX, iX] = 1
kf.H[iZ, iZ] = 1
kf.H[iY, iY] = 1
kf.R *= 5 # KM 
kf.x = np.array([[x, y, z, vx, vy, vz]]).T
kf.u = 0
kf.P = np.eye(6) * 10
saver = Saver(kf)

In [19]:
# # LOAD ALL IMAGES:
# dict = {}
# for img in glob.glob(f"DATA/ephemeris sat/inclination zero/{dt} step size/*"):
#     txt = img             # stringa
#     t = txt.split('_')[1] # numero
#     dict[t] = txt
FILE = pd.read_csv('Pos_Err_d100_quartile_40-60.csv')
H = FILE['H']
LAT = FILE['LAT']
LON = FILE['LON']

In [20]:
mus = []
Zs = []
for idx in range(670):
    if idx == 0:
        # UPDATE
        x,y,z = spherical2cartesian(H[idx],LAT[idx],LON[idx]) # measurement
        Z = np.array([x,y,z])
        kf.update(Z)
        Zs.append(Z)
        saver.save()

    else:    
        kf.predict()
        # UPDATE
        if H[idx] > 0:
            x,y,z = spherical2cartesian(H[idx],LAT[idx],LON[idx]) # measurement
            Z = np.array([x,y,z])
            kf.update(Z)
            saver.save()
            Zs.append(Z)
            x,y,z = 0, 0, 0                             # reset
        else:
            saver.save()
            Zs.append(np.array([np.NaN,np.NaN,np.NaN]))


    mus.append(kf.x)

In [21]:
x_pred = []
for mu in mus:
    x = mu[0]
    x_pred.append(x)
x_true = real_X[:len(x_pred)]

y_pred = []
for mu in mus:
    y = mu[1]
    y_pred.append(y)
y_true = real_Y[:len(y_pred)]

z_pred = []
for mu in mus:
    z = mu[2]
    z_pred.append(z)
z_true = real_Z[:len(z_pred)]
####################################################################
######################## - MEAS - ##################################
X_m = []
for i in range(len(Zs)):
    X_m.append(Zs[i][0])

Y_m = []
for i in range(len(Zs)):
    Y_m.append(Zs[i][1])

Z_m = []
for i in range(len(Zs)):
    Z_m.append(Zs[i][2])

In [22]:
X = saver.x
P = saver.P
P_arr = np.array(P)
Pxx, Pyy, Pzz = [], [], []
for i in range(len(P)):
    Pxx.append(P[i][0,0])
    Pyy.append(P[i][1,1])
    Pzz.append(P[i][2,2])
Pxx, Pyy, Pzz = np.array(Pxx), np.array(Pyy), np.array(Pzz)
Pxx, Pyy, Pzz = np.sqrt(Pxx), np.sqrt(Pyy), np.sqrt(Pzz)

In [28]:
lw = 0.8
S = 4
plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
# plt.subplot(311)
plt.plot(x_true, 'r', linewidth=lw)
plt.plot(Pxx, '--r', linewidth=lw)
plt.plot(-Pxx, '--r', linewidth=lw)
plt.plot(x_pred, '-k', linewidth=lw)
plt.scatter(range(len(X_m)),X_m, s=S)
plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('X [Km]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, : ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])
plt.show()




plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
# plt.subplot(312)
plt.plot(y_true, 'r', linewidth=lw)
plt.plot(Pyy, '--r', linewidth=lw)
plt.plot(-Pyy, '--r', linewidth=lw)
plt.plot(y_pred, '-k', linewidth=lw)
plt.scatter(range(len(X_m)),Y_m, s=S)
plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('Y [Km]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, : ])
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])
plt.show()


plt.figure(dpi=250, tight_layout=True)
plt.subplot(211)
# plt.subplot(313)
plt.plot(z_true, 'r', linewidth=lw)
plt.plot(Pzz, '--r', linewidth=lw)
plt.plot(-Pzz, '--r', linewidth=lw)
plt.plot(z_pred, '-k', linewidth=lw)
plt.scatter(range(len(Z_m)),Z_m, s=S)
plt.legend(['True Position','$\pm\sigma$','_nolegend_','Position estimated by KF','measurements'])
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('Z [Km]')
plt.show()

plt.subplot(212)
Img = cv2.imread('4.bmp')
plt.imshow(Img[235:335, : ])
plt.axis(False)
# plt.yticks([0,50,100],[f'5°',f'0°',f'-5°'])
# plt.xticks([0,int(1665/20),int(1665/20*2),int(1665/20*3),int(1665/20*4),int(1665/20*5),int(1665/20*6),int(1665/20*7),int(1665/20*8),int(1665/20*9),1665//2],[f'{-147+18*10}°',f'{-147+18*11}°',f'{-147+18*12}°',f'{-147+18*13}°',f'{-147+18*14}°',f'{-147+18*15}°',f'{-147+18*16}°',f'{-147+18*17}°',f'{-147+18*18}°',f'{-180+15}°',f'{-147}°'])
plt.show()

In [ ]:
#plt.figure(dpi=150)
#plt.plot(x_pred, '-k')
#plt.scatter([int(i*MES) for i in range(int(N/MES))], [Xs[i*MES] for i in range(int(N/MES))])
#plt.plot(x_true, 'r')
#plt.legend(['pred','true','meas'])
#plt.xlim([1000-1,1000+1])
#plt.ylim([100-5,100+5])
#plt.xlabel('Time')
#plt.ylabel('X')
#plt.show()

In [ ]:
xs = saver.x
xs

In [17]:
plt.figure(dpi=160, tight_layout=True, figsize=(10,10))
lw = 0.8

plt.subplot(311)
x_pred = np.array(x_pred)
x_true = np.array(x_true)
diff = []
for x,y in zip(x_pred,x_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Error - x')
plt.plot(diff, '-k', linewidth=lw)
plt.ylim([-0.2,2.5])
plt.xlim([0,620])
plt.ylabel('Km')
plt.show()

plt.subplot(312)
y_pred = np.array(y_pred)
y_true = np.array(y_true)
diff = []
for x,y in zip(y_pred,y_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Error - y')
plt.plot(diff, '-k', linewidth=lw)
plt.xlim([0,620])
plt.ylim([-0.2,2.5])
plt.ylabel('Km')
plt.show()

plt.subplot(313)
z_pred = np.array(z_pred)
z_true = np.array(z_true)
diff = []
for x,y in zip(z_pred,z_true):
    d = abs(x - y)
    diff.append(d)
plt.title('Error - z')
plt.xlim([0,620])
plt.ylim([-0.2,2])
plt.plot(diff, '-k', linewidth=lw)
plt.xlabel(f'Step Size: {dt}')
plt.ylabel('Km')
plt.show()